In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


# Lab 4: Fire and Tree Mortality

# About the data
Wildfires are increasingly frequent and severe due to climate change. Predicting tree mortality following a wildfire is critical for forest management, ecosystem recovery, and carbon sequestration planning. In this lab, we will build a logistic regression model to predict the probability of tree mortality one year after a wildfire

The database we'll be working with today includes observations of individual trees involved in prescribed fires and wildfires occurring over 35 years, from 1981 to 2016. It is drawn from a fire and tree mortality database from the US Forest Service (see data description for the full database here: [link](https://www.nature.com/articles/s41597-020-0522-7#Sec10)).

The target variable we'll use is `yr1status`, which is a binary variable (0=alive, 1=dead).  This tells us if a tree has died one year after a fire event.

The features we'll use are `YrFireName`, `Times_burned`, `Species`, `Genus_species`,
    `DBH_cm`, `HT_m`, `CR_pre`, and `CR_post`.

## Step 1: Check the metadata

Look at the metadata and provide a description on what each variable represents in the Description column below.


| Feature                     | Description                                                                                   |
|-----------------------------|-----------------------------------------------------------------------------------------------| 
| yr1status                   |Tree status in year 1 post-fire. (0=alive, 1=dead).
| YrFireName                  |Year and name of the fire.
| Times_burned                |The number of times this tree was burned.
| Species                     |Specific epithet
| Genus_species               |Genus and species of tree
| DBH_cm                      |Diameter at breast height rounded to nearest 0.1 cm
| HT_m                        |Pre-fire tree height rounded to nearest 0.01 m 
| CR_pre                      |Pre-fire crown ratio.Crown length divided by height
| CR_post                     |Post-fire live crown ratio

## Step 2: Fetch  data
Read in the data set and filter to retain only the variables of interest.  Then check for incomplete observations and remove any rows containing NaNs.  How many observations does that leave us with? **Print your answer.**

In [14]:
# Load the dataset
trees_dat = pd.read_csv('/courses/EDS232/Data/FTM_trees.csv')

/tmp/ipykernel_1203371/1765753220.py:2: DtypeWarning: Columns (4,5,6,7,10,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  trees_dat = pd.read_csv('/courses/EDS232/Data/FTM_trees.csv')


In [11]:
print(trees_dat.columns)
print(trees_dat.dtypes)

Index(['YrFireName', 'Species', 'Dataset', 'Times_burned', 'ID', 'Plot',
       'TreeNum', 'Unit', 'Genus', 'Species_name', 'Subspecies',
       'Genus_species', 'DBH_cm', 'DRC_cm', 'HT_m', 'HCB_pre', 'CR_pre', 'CC',
       'VR', 'CL_m', 'yr0status', 'yr1status', 'yr2status', 'yr3status',
       'yr4status', 'yr5status', 'yr6status', 'yr7status', 'yr8status',
       'yr9status', 'yr10status', 'HCB_post', 'CR_post', 'CSH_m', 'CLS_m',
       'CLK_m', 'CLS_percent', 'CLK_percent', 'CVS_percent', 'CVK_percent',
       'CVC_percent', 'CBS', 'BCHA_m', 'BCHM_m', 'BCH_percent', 'BCA', 'CKR',
       'GCA', 'GIR_percent', 'DMR', 'BTL', 'AB', 'DFB', 'FEN', 'IPS', 'MPB',
       'RPB', 'RTB', 'SB', 'WPB', 'WB', 'SPB', 'CVS_percent_source',
       'CVK_percent_source'],
      dtype='object')
YrFireName             object
Species                object
Dataset                object
Times_burned            int64
ID                     object
                       ...   
WPB                   float64
W

In [8]:
trees_dat.head()

,YrFireName,Species,Dataset,Times_burned,ID,Plot,TreeNum,Unit,Genus,Species_name,...,IPS,MPB,RPB,RTB,SB,WPB,WB,SPB,CVS_percent_source,CVK_percent_source
0,2006 - Tripod,2TREE,Prichard,1,NaN,188,15.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN
1,2006 - Tripod,2TREE,Prichard,1,NaN,74,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN
2,2006 - Tripod,2TREE,Prichard,1,NaN,193,22.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN
3,2006 - Tripod,2TREE,Prichard,1,NaN,126,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN
4,2006 - Tripod,2TREE,Prichard,1,NaN,113,11.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,NaN


In [16]:
trees_dat = trees_dat.loc[:, ['YrFireName', 'Times_burned', 'Species', 'Genus_species', 'DBH_cm', 'HT_m', 'CR_pre', 'CR_post']]

In [18]:
trees_dat = trees_dat.dropna()

In [27]:
print(trees_dat.head())
print(trees_dat.dtypes)

       YrFireName  Times_burned Species   Genus_species  DBH_cm   HT_m  \
500  2003 - Griff             1    ABAM  Abies_amabilis  71.374  41.76   
501  2003 - Griff             1    ABAM  Abies_amabilis  23.622  12.80   
502  2003 - Griff             1    ABAM  Abies_amabilis  46.228  34.75   
503  2003 - Griff             1    ABAM  Abies_amabilis  21.082  23.16   
504  2003 - Griff             1    ABAM  Abies_amabilis  24.384  26.21   

     CR_pre  CR_post  
500    0.84     0.74  
501    0.60     0.57  
502    0.75     0.59  
503    0.38     0.38  
504    0.42     0.42  
YrFireName        object
Times_burned       int64
Species           object
Genus_species     object
DBH_cm           float64
HT_m             float64
CR_pre           float64
CR_post          float64
dtype: object


## Step 3: Data Preprocessing
1. We recode categorical predictors to zero-based integer form because most machine learning models, including logistic regression, cannot work directly with categorical data represented as strings or labels. Instead, models require numerical input. Let's do that here. 


In [63]:
#factorize, convert dtypes
trees_dat['yrfirename_int'] = pd.factorize(trees_dat['YrFireName'])[0]
trees_dat['species_int'] = pd.factorize(trees_dat['Species'])[0]
trees_dat['genus_species_int'] = pd.factorize(trees_dat['Genus_species'])[0]

print(trees_dat.dtypes)

YrFireName             object
Times_burned            int64
Species                object
Genus_species          object
DBH_cm                float64
HT_m                  float64
CR_pre                float64
CR_post               float64
YrFireName_encoded      int64
YrFireName_int          int64
species_int             int64
yrfirename_int          int64
genus_species_int       int64
dtype: object


In [64]:
print(trees_dat)

            YrFireName  Times_burned Species      Genus_species  DBH_cm  \
500       2003 - Griff             1    ABAM     Abies_amabilis  71.374   
501       2003 - Griff             1    ABAM     Abies_amabilis  23.622   
502       2003 - Griff             1    ABAM     Abies_amabilis  46.228   
503       2003 - Griff             1    ABAM     Abies_amabilis  21.082   
504       2003 - Griff             1    ABAM     Abies_amabilis  24.384   
...                ...           ...     ...                ...     ...   
169332  2003 - B and B             1    TSME  Tsuga_mertensiana  32.512   
169333  2003 - B and B             1    TSME  Tsuga_mertensiana  24.892   
169334  2003 - B and B             1    TSME  Tsuga_mertensiana  32.258   
169335  2003 - B and B             1    TSME  Tsuga_mertensiana  31.750   
169336  2003 - B and B             1    TSME  Tsuga_mertensiana  28.956   

         HT_m  CR_pre  CR_post  YrFireName_encoded  YrFireName_int  \
500     41.76    0.84     0.7

2. Then we'll split into training and test data and scale for coefficient interpretability.  Recall that we use the training features to calculate our scaling parameters (mean and standard deviation) and apply the scaling to those training features (`scaler.fit_transform`) and then apply the scaling to the features in the test data as well (`scaler.transform`).


In [ ]:

...

3. How many training/test observations do we have? Print your answer in the cell below. 

In [ ]:
...

# Verify the training and testing set size
print("Training set observations:", )
print("Testing set observations:",)

## Step 4: Train a Logistical Model
Create a classifier using `LogisticRegression()` and fit it on the training data.  Then assess the model's accuracy on the training set by making predictions on the training data.  Calculate and **print** the accuracy of your model on the training set. 

In [ ]:
...
print(f"Training Accuracy: ")

## Step 5: Test Set Predictions and Model Evaluation
Now let's take our trained logistic classifier and make predictions on the test set. Calculate the accuracy and confusion matrix. Then use `sns.heatmap` for improved confusion matrix visualization.

In [ ]:
...

# Print evaluation metrics
print(f"Accuracy: ")


#Plot confusion matrix
...

## Step 6: Logistic Classifier Evaluation
How did your model perform on the unseen data? 
Does your model perform differently on observations of trees that survived vs trees that died?
Is there a class imbalance in this data set?

*Your answer here*

## Step 7: What about a Dummy?
What do you think would happen if we built a model that always predicts the majority class (dead trees)? How would its accuracy compare to your logistic regression model?

*Your answer here*

Let's go ahead and do it: use `DummyClassifier()` with the appropriate value for the 'strategy' parameter to train a majority classifier.  Then calculate this model's accuracy on the training data.

In [ ]:
...

# Print accuracy and confusion matrix results
print(f"Dummy Accuracy: ")
print("\nDummy Confusion Matrix:")


# Step 8: ROCs and AUCs
Our two models have similar accuracy, but is that all there is to this story?  Let's dig a little deeper on the comparison of our logistic and dummy classifiers by examining the associated receiver-operator characteristic (ROC) curves. Calculate the area under the curve (AUC) for both models.

In [ ]:
# Logistic classifier AUC
...
print(f"Logistic AUC: ")

In [ ]:
# Dummy classifier AUC
...
print(f"Dummy AUC: ")

# Step 9: Plot dummy and logistic model ROC curves
Now using the outputs from `roc_curve()`, plot the ROC curves for both models on the same plot.  Make sure to use appropriate labels in the legend.

How do the two models compare on AUC?  What are the implications for evaluating classifiers based on accuracy of their predictions?

*Your answer here*

# Step 10: Final interpretation

Identifying the most important features in a model can guide decision-making. For instance, in our dataset, highly important features might indicate key factors affecting tree survival after a fire. We will calculate the feature importance by examining the coefficients of our logistic regression model.

In [ ]:
...

# Print the sorted feature importance
print(importance_df)

Which are the most important features in our model (reference the metadata to help answer this)? Can you think of any implications for forest management or conservation strategy?

*Your answer here*